## Interpolate input weather data to dt = 0.5 to match amr wind dt ##

In [ ]:
# Most recently tested against PySAM 4.2.0

# NOTE: at this stage this notebook is not running and needs to be updated

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PySAM.Pvwattsv7 as pvwatts

### Read sample data

In [ ]:
data = pd.read_csv("NonAnnualSimulation-sample_data.csv")
data["Timestamp"] = pd.DatetimeIndex(pd.to_datetime(data["Timestamp"], utc=True))
data = data.set_index("Timestamp")
data = data[data.index > "2018-05-10 12:30:00+0000"]  # select only after 10am UTC

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data_resampled = data.resample("0.5S").asfreq()
data_resampled_filled = data_resampled.interpolate()
data_resampled_filled.head()

In [ ]:
# data['SRRL BMS Direct Normal Irradiance (W/m²_irr)'].plot()
# data_resampled['SRRL BMS Direct Normal Irradiance (W/m²_irr)'].plot(style='.-')
# plt.legend(["data", "resampled"])
# plt.ylabel("Energy (kW)")

for col in data.columns:
    fig, ax = plt.subplots(figsize=[6, 3], dpi=250)
    ax.plot(data[col], "-", label="data")
    # ax.plot(data_resampled['SRRL BMS Direct Normal Irradiance (W/m²_irr)'],'.', label='resampled')
    ax.plot(data_resampled_filled[col], "--", label="filled")
    ax.set_ylabel(col)
    ax.legend()

In [ ]:
data = data_resampled_filled
# data = test_data_interp

weather_data = np.array(
    [
        data.index.year,
        data.index.month,
        data.index.day,
        data.index.hour,
        data.index.minute,
        data["SRRL BMS Direct Normal Irradiance (W/m²_irr)"],
        data["SRRL BMS Diffuse Horizontal Irradiance (W/m²_irr)"],
        data["SRRL BMS Global Horizontal Irradiance (W/m²_irr)"],
        data["SRRL BMS Wind Speed at 19' (m/s)"],
        data["SRRL BMS Dry Bulb Temperature (°C)"],
    ]
)

### Set model parameters

In [ ]:
sys_design = {
    "ModelParams": {
        "SystemDesign": {
            "array_type": 2.0,
            "azimuth": 180.0,
            "dc_ac_ratio": 1.08,
            "gcr": 0.592,
            "inv_eff": 97.5,
            "losses": 15.53,
            "module_type": 2.0,
            "system_capacity": 720,
            "tilt": 0.0,
        },
        "SolarResource": {},
    },
    "Other": {"lat": 39.7442, "lon": -105.1778, "elev": 1829},
}

model_params = sys_design["ModelParams"]
elev = sys_design["Other"]["elev"]
lat = sys_design["Other"]["lat"]
lon = sys_design["Other"]["lon"]
tz = data.index[0].utcoffset().total_seconds() / 60 / 60

### Make predictions

In [ ]:
system_model = pvwatts.new()
system_model.assign(model_params)

solar_resource_data = {
    "tz": tz,  # timezone
    "elev": elev,  # elevation
    "lat": lat,  # latitude
    "lon": lon,  # longitude
    "year": tuple(weather_data[0]),  # year
    "month": tuple(weather_data[1]),  # month
    "day": tuple(weather_data[2]),  # day
    "hour": tuple(weather_data[3]),  # hour
    "minute": tuple(weather_data[4]),  # minute
    "dn": tuple(weather_data[5]),  # direct normal irradiance
    "df": tuple(weather_data[6]),  # diffuse irradiance
    "gh": tuple(weather_data[7]),  # global horizontal irradiance
    "wspd": tuple(weather_data[8]),  # windspeed
    "tdry": tuple(weather_data[9]),  # dry bulb temperature
}

system_model.SolarResource.assign({"solar_resource_data": solar_resource_data})
system_model.AdjustmentFactors.assign({"constant": 0})

system_model.execute()
out = system_model.Outputs.export()

ac = np.array(out["ac"]) / 1000
dc = np.array(out["dc"]) / 1000

predictions = pd.DataFrame(
    {"ac": ac, "dc": dc, "aoi": np.array(out["aoi"])}, columns=["ac", "dc", "aoi"]
)
predictions = predictions.set_index(data.index.copy())

In [ ]:
fig, ax = plt.subplots()
ax.plot(predictions["ac"], "-", label="Predicted energy")
ax.plot(data["Energy (kW)"], "-", label="Actual energy")
ax.legend()
plt.ylabel("Energy (kW)")

In [ ]:
fig, ax = plt.subplots()
ax.plot(predictions["aoi"])
plt.ylabel("Angle of Incidence")

Looks good so let's save the interpolated data as a csv

In [ ]:
data.to_csv("NonAnnualSimulation-sample_data-interpolated.csv")
data_resampled_filled.to_csv("NonAnnualSimulation-sample_data-interpolated-daytime.csv")

In [ ]:
test_data_interp = pd.read_csv("NonAnnualSimulation-sample_data-interpolated.csv")
test_data_interp["Timestamp"] = pd.DatetimeIndex(
    pd.to_datetime(test_data_interp["Timestamp"], format="ISO8601", utc=True)
)
test_data_interp = test_data_interp.set_index("Timestamp")
test_data_interp.head()

In [ ]:
test_data_interp.info()

In [ ]:
data.info()

In [ ]:
data.index[-1] - data.index[0]

In [ ]:
# plot original data vs interpolated data read in from csv
for col in data.columns:
    fig, ax = plt.subplots(figsize=[6, 3], dpi=250)
    ax.plot(data[col], "-", label="original")
    # ax.plot(data_resampled['SRRL BMS Direct Normal Irradiance (W/m²_irr)'],'.', label='resampled')
    ax.plot(test_data_interp[col], "--", label="interp")
    ax.set_ylabel(col)
    ax.legend()